In [1]:

import os
import json
import pandas as pd
import geopandas as gpd
import re

#### Read the downloaded/nonspatial and spatial data

In [2]:

os.chdir('..')
home_dir = os.getcwd()
downloaded_data = r"downloaded-data"


In [3]:
# replace... blds_continents_data_18-23_ai.json; blds_africa_states_18-23_ai.json; blds_northamerica_states_18-23_ai.json
with open(f"{downloaded_data}\\blds_northamerica_states_18-23_ai.json", "r") as f:
    nonspatial_d = json.load(f)
# nonspatial_d



In [4]:
os.getcwd()

'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats'

In [26]:
shp = r"shp"
# change name of the layer - africa-states-borders.shp; northamerica-states-borders.shp
s_df = gpd.read_file(f"{shp}/northamerica-states-borders.shp")
s_df.head()


,iso3,status,color_code,name,continent,region,iso_3166_1,french_sho,geometry
0,PAN,Member State,PAN,Panama,Americas,Central America,PA,Panama,"MULTIPOLYGON (((-81.67847 7.38861, -81.64944 7..."
1,BLZ,Member State,BLZ,Belize,Americas,Central America,BZ,Belize,"MULTIPOLYGON (((-87.81903 17.42902, -87.83153 ..."
2,NIC,Member State,NIC,Nicaragua,Americas,Central America,NI,Nicaragua,"POLYGON ((-83.13185 14.99298, -83.19667 14.956..."
3,SLV,Member State,SLV,El Salvador,Americas,Central America,SV,El Salvador,"POLYGON ((-89.34831 14.43198, -89.33216 14.410..."
4,CAN,Member State,CAN,Canada,Americas,Northern America,CA,Canada,"MULTIPOLYGON (((-66.87528 44.61944, -66.88612 ..."


#### Edit the obtained data into desired shape to create a DataFrame

In [6]:
d = {
    "feature": [],
    "source": [],
    "timestamp": [],
    "value": []
}


In [7]:
for cont in nonspatial_d:
    for dic1 in cont:
        feature, source = dic1["groupByObject"][0], dic1["groupByObject"][1]
        for res in dic1["result"]:
            d["feature"].append(feature)
            d["source"].append(source)
            d["timestamp"].append(res["timestamp"])
            d["value"].append(res["value"])

ns_df = pd.DataFrame(d)
# ns_df

In [8]:
# extract only the year
ns_df["year"] = pd.to_datetime(ns_df["timestamp"]).dt.year

In [9]:
ns_df

,feature,source,timestamp,value,year
0,0,remainder,2018-01-01T00:00:00Z,27034.0,2018
1,0,remainder,2019-01-01T00:00:00Z,41003.0,2019
2,0,remainder,2020-01-01T00:00:00Z,49624.0,2020
3,0,remainder,2021-01-01T00:00:00Z,98964.0,2021
4,0,remainder,2022-01-01T00:00:00Z,140847.0,2022
...,...,...,...,...,...
157,9,source=microsoft/BuildingFootprints,2019-01-01T00:00:00Z,0.0,2019
158,9,source=microsoft/BuildingFootprints,2020-01-01T00:00:00Z,0.0,2020
159,9,source=microsoft/BuildingFootprints,2021-01-01T00:00:00Z,0.0,2021
160,9,source=microsoft/BuildingFootprints,2022-01-01T00:00:00Z,0.0,2022


#### Append the names of the regions and numbers to the DataFrame

In [10]:

# replace... geojson-continent-borders, geojson-africa-states; geojson-northamerica-states
geojson_dir = os.path.join(home_dir, "geojson-northamerica-states", "")

geojson_names = []
for (dir_path, dir_names, file_names) in os.walk(geojson_dir):  # give the path
    geojson_names.extend(file_names)
print(geojson_names)
print(os.getcwd())


['0_Panama.geojson', '10_Midway Is..geojson', '11_Jarvis Island.geojson', '12_United States of America.geojson', '13_Mexico.geojson', '1_Belize.geojson', '2_Nicaragua.geojson', '3_El Salvador.geojson', '4_Canada.geojson', '5_Greenland.geojson', '6_Guatemala.geojson', '7_Bermuda.geojson', '8_Costa Rica.geojson', '9_Honduras.geojson']
c:\Users\milan\OneDrive - MUNI\VŠ\PhD\Zahraniční stáž\Work\HeiGIT_notebooks\analysis\ai-assisted-osm-mapping-stats


In [11]:
geojson_names_2 = geojson_names

id_name_dic = {int(re.search(r'^\d+', k)[0]): re.search(r'\_.+\.', v)[0][1:-1]
                for k, v in zip(geojson_names, geojson_names_2)}
id_name_ser = pd.Series(id_name_dic)
id_name_df = pd.DataFrame(id_name_ser, columns=["name"])
id_name_df = id_name_df.reset_index().rename(columns={"index": "id"})
id_name_df["id"] = id_name_df["id"].astype(int)
# id_name_df


In [12]:
# merge df with region names based on ids
ns_df = ns_df.merge(id_name_df, how="left", left_on="feature",
               right_on="id")
ns_df.head(20)


,feature,source,timestamp,value,year,id,name
0,0,remainder,2018-01-01T00:00:00Z,27034.0,2018,0,Panama
1,0,remainder,2019-01-01T00:00:00Z,41003.0,2019,0,Panama
2,0,remainder,2020-01-01T00:00:00Z,49624.0,2020,0,Panama
3,0,remainder,2021-01-01T00:00:00Z,98964.0,2021,0,Panama
4,0,remainder,2022-01-01T00:00:00Z,140847.0,2022,0,Panama
5,0,remainder,2023-01-01T00:00:00Z,158876.0,2023,0,Panama
6,0,source=microsoft/BuildingFootprints,2018-01-01T00:00:00Z,0.0,2018,0,Panama
7,0,source=microsoft/BuildingFootprints,2019-01-01T00:00:00Z,0.0,2019,0,Panama
8,0,source=microsoft/BuildingFootprints,2020-01-01T00:00:00Z,0.0,2020,0,Panama
9,0,source=microsoft/BuildingFootprints,2021-01-01T00:00:00Z,0.0,2021,0,Panama


#### Data preparation and filtering

In [13]:
# get the number of AI Microsoft (source=microsoft/BuildingFootprints) / manually mapped (remainder) buildings for every year
grouped = ns_df.groupby(["source", "year"])["value"].sum()
# grouped

In [14]:
# merge the DataFrame with groupBy DataFrame and compute the percentage
merged_data = ns_df.merge(grouped, on=["source", 'year'], how='left', )
merged_data = merged_data.rename(columns={"value_x":"num_of_blds", "value_y":"total_blds"})
merged_data["percentage"] = (merged_data["num_of_blds"] / merged_data["total_blds"]) * 100
# merged_data.head(10)


In [15]:
# percentage verification
merged_data.groupby(["source", "year"])["percentage"].sum()


source                               year
remainder                            2018    100.0
                                     2019    100.0
                                     2020    100.0
                                     2021    100.0
                                     2022    100.0
                                     2023    100.0
source=microsoft/BuildingFootprints  2018      0.0
                                     2019      0.0
                                     2020    100.0
                                     2021    100.0
                                     2022    100.0
                                     2023    100.0
Name: percentage, dtype: float64

In [16]:
ns_df_rem = merged_data.loc[merged_data["source"] == "remainder", :]
ns_df_ai = merged_data.loc[merged_data["source"] == "source=microsoft/BuildingFootprints", :]


In [17]:
ns_df_rem_sel = ns_df_rem.groupby("name")[["num_of_blds", "year", "percentage", "total_blds"]].last().reset_index()
ns_df_ai_sel = ns_df_ai.groupby("name")[["num_of_blds", "year", "percentage", "total_blds"]].last().reset_index()
ns_df_ai_sel.head()

,name,num_of_blds,year,percentage,total_blds
0,Belize,0.0,2023,0.000000,8504230.0
1,Bermuda,0.0,2023,0.000000,8504230.0
2,Canada,163150.0,2023,1.918457,8504230.0
3,Costa Rica,150.0,2023,0.001764,8504230.0
4,El Salvador,2100.0,2023,0.024694,8504230.0


In [18]:
ns_df_rem_sel.head()

,name,num_of_blds,year,percentage,total_blds
0,Belize,150054.0,2023,0.248825,60304931.0
1,Bermuda,5759.0,2023,0.009550,60304931.0
2,Canada,5656443.0,2023,9.379735,60304931.0
3,Costa Rica,148014.0,2023,0.245443,60304931.0
4,El Salvador,259222.0,2023,0.429852,60304931.0


In [19]:
s_df.head(2)

,iso3,status,color_code,name,continent,region,iso_3166_1,french_sho,geometry
0,PAN,Member State,PAN,Panama,Americas,Central America,PA,Panama,"MULTIPOLYGON (((-81.67847 7.38861, -81.64944 7..."
1,BLZ,Member State,BLZ,Belize,Americas,Central America,BZ,Belize,"MULTIPOLYGON (((-87.81903 17.42902, -87.83153 ..."


In [20]:
merged_d_rem = s_df.merge(ns_df_rem_sel, how="left",
                          left_on="name", right_on="name").rename(columns={"name_y": "added_name"})  # change left_on
merged_d_ai = s_df.merge(ns_df_ai_sel, how="left",
                         left_on="name", right_on="name").rename(columns={"name_y": "added_name"})  # change left_on
merged_d_rem.head(2)


,iso3,status,color_code,name,continent,region,iso_3166_1,french_sho,geometry,num_of_blds,year,percentage,total_blds
0,PAN,Member State,PAN,Panama,Americas,Central America,PA,Panama,"MULTIPOLYGON (((-81.67847 7.38861, -81.64944 7...",158876.0,2023.0,0.263454,60304931.0
1,BLZ,Member State,BLZ,Belize,Americas,Central America,BZ,Belize,"MULTIPOLYGON (((-87.81903 17.42902, -87.83153 ...",150054.0,2023.0,0.248825,60304931.0


In [21]:
merged_d_ai.head(2)

,iso3,status,color_code,name,continent,region,iso_3166_1,french_sho,geometry,num_of_blds,year,percentage,total_blds
0,PAN,Member State,PAN,Panama,Americas,Central America,PA,Panama,"MULTIPOLYGON (((-81.67847 7.38861, -81.64944 7...",2086.0,2023,0.024529,8504230.0
1,BLZ,Member State,BLZ,Belize,Americas,Central America,BZ,Belize,"MULTIPOLYGON (((-87.81903 17.42902, -87.83153 ...",0.0,2023,0.000000,8504230.0


In [22]:
print(merged_d_rem.columns)
print(merged_d_ai.columns)


Index(['iso3', 'status', 'color_code', 'name', 'continent', 'region',
       'iso_3166_1', 'french_sho', 'geometry', 'num_of_blds', 'year',
       'percentage', 'total_blds'],
      dtype='object')
Index(['iso3', 'status', 'color_code', 'name', 'continent', 'region',
       'iso_3166_1', 'french_sho', 'geometry', 'num_of_blds', 'year',
       'percentage', 'total_blds'],
      dtype='object')


#### Export the data into layers

In [27]:
os.getcwd()

'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats'

In [28]:
# change name of the layer
merged_d_rem.to_file("maps/shp/mm_blds_northamerica_states.shp", driver='ESRI Shapefile')


C:\Users\milan\AppData\Local\Temp\ipykernel_24372\1468117952.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_d_rem.to_file("maps/shp/mm_blds_northamerica_states.shp", driver='ESRI Shapefile')


In [29]:
# change name of the layer
merged_d_ai.to_file("maps/shp/ai_blds_northamerica_states.shp", driver='ESRI Shapefile')


C:\Users\milan\AppData\Local\Temp\ipykernel_24372\1974897325.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_d_ai.to_file("maps/shp/ai_blds_northamerica_states.shp", driver='ESRI Shapefile')
